In [ ]:
!pip install wheel setuptools pip --upgrade
!pip install --upgrade openai

  Using cached setuptools-75.1.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
Using cached setuptools-75.1.0-py3-none-any.whl (1.2 MB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 71.0.4
    Uninstalling setuptools-71.0.4:
      Successfully uninstalled setuptools-71.0.4
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [ ]:
import pandas as pd
import json
import random
from openai import OpenAI
import time
import numpy as np
import gzip
import re

API_KEY = ''
client = OpenAI(api_key = API_KEY)
model_id = 'gpt-3.5-turbo-0125' #gpt-3.5-turbo-0125, gpt-4-turbo-2024-04-09

# Replace 'path_to_file.jsonl' with the path to your JSONL file
file_path = 'GSM8K.jsonl'

data = []

# Open the file and read each line
with open(file_path, 'r') as file:
    for line in file:
        # Convert each JSON string into a Python dictionary
        json_dict = json.loads(line)
        data.append(json_dict)

# 'data' is now a list of dictionaries
print(data[11]['question'])
print( re.findall(r'\n#### (.*)', data[11]['answer'])[0] )

samples = []
targets = []
for i in range(10):
    samples.append(data[i]['question'])
    targets.append( re.findall(r'\n#### (.*)', data[i]['answer'])[0] )


In [ ]:
meta_prompt = f"""
You are given a math question statement. Your task is to propose a creative, detailed, and step-by-step algorithm to reformulate and enrich this question statement. The goal is of the algorithm is to perform a thorough engineering on the statement, so that it is easier for a LLM to solve it. Below are some sample math question statements as refrences.

Examples:
- Job Description: '{samples[0]}'; Is the job Fraud or not: '{targets[0]}'
- Job Description: '{samples[1]}'; Is the job Fraud or not: '{targets[1]}'
- Job Description: '{samples[2]}'; Is the job Fraud or not: '{targets[2]}'
- Job Description: '{samples[3]}'; Is the job Fraud or not: '{targets[3]}'
- Job Description: '{samples[4]}'; Is the job Fraud or not: '{targets[4]}'
- Job Description: '{samples[5]}'; Is the job Fraud or not: '{targets[5]}'
- Job Description: '{samples[6]}'; Is the job Fraud or not: '{targets[6]}'
- Job Description: '{samples[7]}'; Is the job Fraud or not: '{targets[7]}'
- Job Description: '{samples[8]}'; Is the job Fraud or not: '{targets[8]}'
- Job Description: '{samples[9]}'; Is the job Fraud or not: '{targets[9]}'

For each step of the algorithm, number and then start it on a new line; you must start each step with '###------- Step 1: ' The proposed algorithm will later be submitted to a LLM for processing.
Important: Do NOT refer to any external database; Do NOT perform the item counting. Do NOT perform normalization. Do NOT perform vector generations. Do NOT propose to draw anything. Do NOT perform similarity checking. Do NOT propose how to train or generate a recommendation system. ONLY propose steps that a LLM can generate on its own!!!
"""

prompts = []
prompt_performances = {}
for i in range(7):

    completion = client.chat.completions.create(
        model = model_id, temperature = 1.0,

        messages=[{"role": "system", "content": "Please come up with a step-by-step, very detailed, clear, and novel algorithm for reformulating and enriching a math question statement for easier LLM processing. Return the steps ONLY!!! NO more than 5 steps. You MUST NOT propose to directly answer the question in the steps!!!"},
                    {"role": "user", "content": meta_prompt}],
        timeout = 1200)

    candidate_prompt = completion.choices[0].message.content
    prompts.append(candidate_prompt)
    print(candidate_prompt)
    print()


In [ ]:
import re

for candidate_prompt in prompts:

    print(candidate_prompt)
    print()

    # Regular expression to split the text into individual algorithms
    algorithm_pattern = r"###------- Step \d: [\w\s]+"

    # Split the text based on the pattern
    split_text = re.split(algorithm_pattern, candidate_prompt)

    # Extract the algorithm headers (for identification)
    headers = re.findall(algorithm_pattern, candidate_prompt)

    # Removing the first empty string from the split if exists (because of the split at the start)
    split_text = [t.strip() for t in split_text if t.strip()]

    # Create a dictionary where each algorithm is stored separately
    algorithms = {headers[i]: split_text[i] for i in range(len(headers))}

    # Display each algorithm separately
    steps = []
    for header, content in algorithms.items():
        steps.append( [header, content] )
        # print(f"{header}:\n{content}\n")

    total_right = 0
    ADE_total_right = 0
    total = 0

    for i in range(11, len(data)):
        question = data[i]['question']
        target = re.findall(r'\n#### (.*)', data[i]['answer'])[0]

        original_prompt = (
            'Given a math question, please solve it by returning the final answer as a number.' +
            f'\n\nQuestion: {question}' +
            "Output format: directly return an algebric number representing the answer; do NOT explain anything. You MUST follow this format: 'The answer is: 34.5'!!!"
        )

        completion = client.chat.completions.create(
            model = model_id, temperature = 0, seed = 0,

            messages=[{"role": "system", "content": 'Please solve the math question presented by outputting the final answer.'},
                        {"role": "user", "content": original_prompt}],
            timeout = 1200)

        ori_answer = completion.choices[0].message.content

        total += 1
        match = re.search(r'[-+]?\d*\.\d+|\d+', ori_answer)
        if match:
            ori_answer = float(match.group())

        if float(target) == float(ori_answer):
            total_right += 1


        overall_with_steps = "Original question statement: " + question + '\n\n'
        for i in range(len(algorithms)):

            reformulation_prompt = f"Please thoroughly reformulate the math question statement based on the following instruction:\n\n{steps[i][0]}\n{steps[i][1]}\n\nQuestion statement to reformulate: {overall_with_steps}."

            completion = client.chat.completions.create(
                    model = model_id, temperature = 0.0, max_tokens = 768,

                    messages=[{"role": "system", "content": "Please reformulate and enrich the math question to be extremely informative and detailed for LLM to interpret better. You are allowed to infer and fill-in unspecified information based on your domain expertise!"},
                                {"role": "user", "content": reformulation_prompt}],
                    timeout = 1200)

            reformulated_history = completion.choices[0].message.content
            # print('Step ' + str(i) + ' Results: ')
            # print(reformulated_history)
            overall_with_steps += steps[i][0] + '\n' + steps[i][1] + reformulated_history + '\n\n'
            # print()
            # print()

        print(overall_with_steps)
        print()

        ADE_prompt = (
            'Given a math question, please solve it by returning the final answer as a number.' +
            f'\n\nQuestion: {overall_with_steps}' +
            "Output format: directly return an algebric number representing the answer; do NOT explain anything. You MUST follow this format: 'The answer is: 34.5'!!!"
        )

        completion = client.chat.completions.create(
            model = model_id, temperature = 0, seed = 0,

            messages=[{"role": "system", "content": 'Please solve the math question presented by outputting the final answer.'},
                        {"role": "user", "content": ADE_prompt}],
            timeout = 1200)

        answer = completion.choices[0].message.content

        match = re.search(r'[-+]?\d*\.\d+|\d+', answer)
        if match:
            answer = float(match.group())

        if float(target) == float(answer):
            ADE_total_right += 1

        print(target, ori_answer, answer)
        print(f'Total: {total}, Total Right: {total_right}, ADE Total Right: {ADE_total_right}')
        print('----------------------------------------------------------------------------------------')
        print()

        if total % 50 == 0:
            if total_right >= ADE_total_right:
                prompt_performances[candidate_prompt] = ADE_total_right / total
                print()
                break

        if total >= 200:
            prompt_performances[candidate_prompt] = ADE_total_right / total
            break
